In [ ]:
import geopandas as gpd
import pandas as pd
import parquet

In [ ]:
quadrangles = gpd.read_file("raw_data/7.5_Minute_Quadrangle_Boundaries-shp")
quadrangles.info()

In [ ]:
leases = gpd.read_file("raw_data/ASLD_-_State_Trust_Parcels/ASLD_-_State_Trust_Parcels.shp")
leases.info()

In [ ]:
leases.plot()

In [ ]:
#First we find all quadrangles that overlap with a lease
outside_quadrangles = gpd.sjoin(left_df=quadrangles, right_df=leases, how='left', predicate="overlaps")
#Next we keep only the quadrangles that did overlap
lease_quadrangles = outside_quadrangles[outside_quadrangles['landnum'].notna()]
lease_quadrangles = outside_quadrangles[quadrangles.keys()]
#Next we keep only the quadrangles that did not overlap
outside_quadrangles = outside_quadrangles[outside_quadrangles['landnum'].isna()]
outside_quadrangles = outside_quadrangles[quadrangles.keys()]
outside_quadrangles.plot()

In [ ]:
outside_quadrangles = outside_quadrangles.to_crs(crs=3857)
lease_quadrangles = lease_quadrangles.to_crs(crs=3857)
leases = leases.to_crs(crs=3857)
outside_quadrangles["nearest_quadrangle"] = outside_quadrangles.geometry
lease_quadrangles["lease_quadrangle"] = lease_quadrangles.geometry

In [ ]:
leases_with_nearest_quadrangle = gpd.sjoin_nearest(left_df=leases, right_df=outside_quadrangles, how='left')
leases_with_nearest_quadrangle = leases_with_nearest_quadrangle.drop(['index_right'], axis=1)
leases_with_nearest_quadrangle = gpd.sjoin_nearest(left_df=leases_with_nearest_quadrangle, right_df=lease_quadrangles, how='left')
leases_with_nearest_quadrangle = leases_with_nearest_quadrangle.drop(['index_right'], axis=1)
leases_with_nearest_quadrangle.info()

In [ ]:
leases_with_nearest_quadrangle.lease_quadrangle.plot()

In [ ]:
leases_with_nearest_quadrangle.nearest_quadrangle.plot()

In [ ]:
leases.plot()

In [ ]:
leases_with_nearest_quadrangle.to_crs("EPSG:4326").to_parquet("our_data/leases_with_nearest_quadrangles.parquet")

In [ ]:
leases